<a href="https://colab.research.google.com/github/ihshin227/tensorlite_convert_test/blob/master/Keras_convlstm_convert_to_tensorflow_lite_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.ConvLSTM2D(filters=40, kernel_size=(3,3), input_shape=(40, 40, 3, 1), padding='same', return_sequences=True))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ConvLSTM2D(filters=40, kernel_size=(3,3), padding='same', return_sequences=True))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ConvLSTM2D(filters=40, kernel_size=(3,3), padding='same', return_sequences=True))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ConvLSTM2D(filters=40, kernel_size=(3,3), padding='same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Reshape((-1, 4800)))
model.add(tf.keras.layers.Dense(units=40))
model.compile(loss='binary_crossentropy', optimizer='adadelta')

In [0]:
import pylab as plt

inputs = np.random.random([32, 40, 40, 3, 1]).astype(np.float32)
targets = np.random.random([32, 40]).astype(np.float32)

model.fit(inputs, targets, epochs=30)

Epoch 1/30
1/1 [==============================] - 0s 1ms/step - loss: 6.0265
Epoch 2/30
1/1 [==============================] - 0s 1ms/step - loss: 6.0056
Epoch 3/30
1/1 [==============================] - 0s 1ms/step - loss: 5.9890
Epoch 4/30
1/1 [==============================] - 0s 2ms/step - loss: 5.9906
Epoch 5/30
1/1 [==============================] - 0s 1ms/step - loss: 5.9848
Epoch 6/30
1/1 [==============================] - 0s 1ms/step - loss: 5.9821
Epoch 7/30
1/1 [==============================] - 0s 2ms/step - loss: 5.9782
Epoch 8/30
1/1 [==============================] - 0s 2ms/step - loss: 5.9668
Epoch 9/30
1/1 [==============================] - 0s 1ms/step - loss: 5.9655
Epoch 10/30
1/1 [==============================] - 0s 1ms/step - loss: 5.9613
Epoch 11/30
1/1 [==============================] - 0s 1ms/step - loss: 5.9482
Epoch 12/30
1/1 [==============================] - 0s 1ms/step - loss: 5.9439
Epoch 13/30
1/1 [==============================] - 0s 1ms/step - loss: 5.

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

tflite_results = interpreter.get_tensor(output_details[0]['index'])
tf_results = model(tf.constant(input_data))

ConverterError: ignored